In [69]:
import sys
sys.path.insert(0, '../../style_generation_pipeline')

In [70]:
import pandas as pd
import json
import sklearn
import glob
import pickle

pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [71]:
from data import *

In [72]:
path='/mnt/swordfish-pool2/milad/hiatus-data/phase_1'

In [73]:
raw_data = pd.read_json(path + '/raw_data.jsonl', lines=True)

In [74]:
len(raw_data)

42734

In [8]:
raw_data.head()

documentID  \
0  2d35aeeb-c85f-58d3-a1f6-7fdda79681ef   
1  2f22836a-f300-5213-9207-b5b16d9d7267   
2  feeec1b1-5e1e-5d45-8541-2989a52ae090   
3  c553e4aa-4ce8-5828-81a8-7ce479e716c1   
4  4ff0c234-4ad1-57e4-9368-77646d7aa471   

                                authorIDs  \
0  [f2c6d500-808c-5905-8bb9-f67d93a5496a]   
1  [f2c6d500-808c-5905-8bb9-f67d93a5496a]   
2  [f2c6d500-808c-5905-8bb9-f67d93a5496a]   
3  [f2c6d500-808c-5905-8bb9-f67d93a5496a]   
4  [c101dce2-422f-5807-bf88-f79904f400dc]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [8]:
#sample a set to generate style features from
sample_raw_data = raw_data.sample(10000)

In [10]:
output_path = path + '/raw_data_sample.jsonl'

with open(output_path, "w") as f:
    f.write(sample_raw_data.to_json(orient='records', lines=True))

### Prepare authors for clustering:

In [40]:
sample_training_ds = pd.read_json(path + '/raw_data_sample.jsonl', lines=True)

In [42]:
len(sample_training_ds)

10000

In [50]:
sample_training_ds.columns

Index(['documentID', 'authorIDs', 'fullText', 'spanAttribution',
       'isForeground', 'dateCollected', 'publiclyAvailable', 'collectionNum',
       'source', 'deidentified', 'languages', 'lengthWords', 'dateCreated',
       'timeCreated', 'sourceSpecific', 'machineAuthored'],
      dtype='object')

In [51]:
sample_authors = set([x[0] for x in sample_training_ds.authorIDs.tolist()])

In [43]:
# Cluster of the training and dev
training_df,_,_ = get_aa_data_from_original_format(path + '/train-test-dev-split/train/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/data/hrs_release_08-14-23_crossGenre-combined_TA2_input',
                                              path + '/train-test-dev-split/train/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/groundtruth/hrs_release_08-14-23_crossGenre-combined_TA2')

dev_df,_,_ = get_aa_data_from_original_format(path + '/train-test-dev-split/dev/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/data/hrs_release_08-14-23_crossGenre-combined_TA2_input',
                                              path + '/train-test-dev-split/dev/official-query-candidate-format/TA2/hrs_release_08-14-23_crossGenre-combined/groundtruth/hrs_release_08-14-23_crossGenre-combined_TA2')

In [44]:
training_df.reset_index(inplace=True)
dev_df.reset_index(inplace=True)

In [56]:
filtered_training_df = training_df[training_df.authorID.isin(sample_authors)]
filtered_dev_df = dev_df[dev_df.authorID.isin(sample_authors)]

In [57]:
len(filtered_dev_df)

291

In [55]:
len(filtered_training_df)

2121

In [62]:
training_df.to_json(path + "/training_authors.json")
dev_df.to_json(path + "/valid_authors.json")

### Generated Style Features:

In [18]:
#df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/refined_and_aggregated_features_final.csv')
#df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explanation_10k/style_corpus.csv')
df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explanation_10k/filtered/refined_and_aggregated_features_final.csv')

In [19]:
df.head()

,final_attribute_name,original_attribute_name,documentID,shortend_attribute_name.v1,shortend_attribute_name.v2,aggregated_name
0,style is casual,"The author employs a conversational tone, using first-person pronouns and colloquial expressions.",71998606-5f55-54c4-a066-c26c309fdede,The author uses a casual tone in their writing.,The writing style is casual.,The writing style is casual.
1,clear sentence structures,"The author uses simple sentence structures, with an average of 15-20 words per sentence.",da7233ef-72e8-51d8-a884-e1e221866070,The author uses straightforward sentence structures.,The author uses clear sentence structures.,The author uses clear sentence structures.
2,diverse sentence structures,"The author employs a mix of short and medium-length sentences, with no extremely long sentences.",da7233ef-72e8-51d8-a884-e1e221866070,The author uses a variety of sentence lengths in their writing.,The writer employs diverse sentence structures in their work.,The writer employs diverse sentence structures in their work.
3,passive voice,The author favors active voice over passive voice.,da7233ef-72e8-51d8-a884-e1e221866070,Rewritten: Active voice is preferred over passive voice.,Rewritten: Choose active voice over passive voice.,Rewritten: Choose active voice over passive voice.
4,active voice,The author favors active voice over passive voice.,da7233ef-72e8-51d8-a884-e1e221866070,Rewritten: Active voice is preferred over passive voice.,Rewritten: Choose active voice over passive voice.,Rewritten: Choose active voice over passive voice.


In [20]:
df['final_attribute_name'].nunique()

358

In [21]:
df['final_attribute_name'].value_counts()

diverse sentence structures                4221
consistent verb tense                      3023
simple sentence structures                 2774
style is formal                            1643
different sentence structures              1249
various sentence types                     1200
technical language                         1000
varied sentence lengths                     960
complex sentence structures                 879
various sentence structures                 873
casual language                             821
clear sentence structures                   814
formal tone                                 812
tense usage                                 794
style is professional                       705
better rhythm                               502
different tenses                            363
extra information                           356
specialized language                        327
subordinate clauses                         290
visual impact                           

#### Merge the Gram2vec style feats to our LLM-based style feats:

In [20]:
gram2vec = json.load(open('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/gram2vec_top_features.jsonl'))

In [21]:
gram2vec = {x['documentID'] : x['gram2vec_feats'] for x in gram2vec}

In [24]:
gram2vec_feats = set([x for feats in gram2vec.values() for x in feats])

In [26]:
print(len(gram2vec_feats))

638


In [27]:
len(gram2vec)

53332

In [28]:
agg_df = df.groupby('documentID').agg({'final_attribute_name': lambda x: list(x)}).reset_index()

In [29]:
agg_df['final_attribute_name'] = agg_df.apply(lambda x: x['final_attribute_name'] + gram2vec[x['documentID']], axis=1)

In [30]:
agg_df.head()

,documentID,final_attribute_name
0,000d37d2-efa7-583f-b302-f0d15f552d39,"[long sentences, complex sentences, subordinate clauses, visual interest, pos_bigrams:AUX NUM, func_words:'ll, morph_tags:Degree=Cmp, morph_tags:PronType=Dem, pos_bigrams:ADV PART, pos_bigrams:ADV VERB, pos_bigrams:AUX ADV, func_words:these, pos_bigrams:NUM NOUN, morph_tags:VerbType=Mod]"
1,001fcc11-83c6-5dd1-b4a7-17b18421193a,"[complex sentence structures, multiple clauses, vivid imagery, pos_bigrams:AUX X, func_words:his, pos_bigrams:ADP PRON, morph_tags:Gender=Masc, func_words:himself, pos_bigrams:DET PRON, func_words:themselves, morph_tags:Poss=Yes, pos_bigrams:PUNCT PART, dep_labels:poss]"
2,002adbbe-d084-5a2e-a5a8-2f7b2f34e1e1,"[simple sentence structures, long sentences, consistent verb tense, present tense, declarative sentences, limited range, func_words:again, pos_bigrams:INTJ PRON, letters:D, pos_bigrams:PRON PROPN, punctuation:!, pos_bigrams:PROPN ADV, pos_bigrams:ADP PART, pos_bigrams:ADV NOUN, pos_bigrams:PROPN ADP, func_words:been]"
3,002ae3ff-0325-5dce-b23a-dfde42aa7cb6,"[complex sentence structures, precise vocabulary, formal tone, long sentences, punctuation:!, pos_bigrams:PROPN AUX, letters:H, letters:D, pos_bigrams:PROPN ADV, pos_bigrams:PROPN NOUN, pos_bigrams:DET PROPN, func_words:just, pos_bigrams:PRON PROPN, func_words:as]"
4,002df921-62ed-52a7-a646-eb22979d5ace,"[complex sentence structures, multiple clauses, additional information, subordinate clauses, passive voice, dep_labels:csubj, func_words:most, pos_bigrams:NUM ADJ, pos_bigrams:SCONJ DET, morph_tags:Gender=Neut, pos_bigrams:ADJ DET, pos_bigrams:PUNCT SCONJ, pos_bigrams:AUX CCONJ, morph_tags:VerbForm=Ger, pos_bigrams:SCONJ ADV]"


In [31]:
joined_df = agg_df.explode(['final_attribute_name'])

In [32]:
joined_df.head()

,documentID,final_attribute_name
0,000d37d2-efa7-583f-b302-f0d15f552d39,long sentences
0,000d37d2-efa7-583f-b302-f0d15f552d39,complex sentences
0,000d37d2-efa7-583f-b302-f0d15f552d39,subordinate clauses
0,000d37d2-efa7-583f-b302-f0d15f552d39,visual interest
0,000d37d2-efa7-583f-b302-f0d15f552d39,pos_bigrams:AUX NUM


In [33]:
joined_df.to_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/gram2vec_and_llm_10k_based_style_features.csv')

In [35]:
joined_df.final_attribute_name.nunique()

857

### Consturcted Interpretable space:

In [75]:
#df = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/training_authors.pkl')
df = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explanation_10k/training_authors.pkl')

In [76]:
df.cluster_label.nunique()

157

In [77]:
#vector_to_style_dist = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/interpretable_space.pkl')
vector_to_style_dist = pd.read_pickle('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explanation_10k/interpretable_space.pkl')

In [78]:
len(vector_to_style_dist)

157

In [79]:
clusters_and_top10_feats = {key : {y[0]: y[1] for y in sorted(x[1].items(), key=lambda i:-i[1])[:10]} for key, x in vector_to_style_dist.items()}

In [80]:
clusters_and_top10_feats

{-1: {'simple sentence structures': 826.0752164017492,
  'diverse sentence structures': 796.94704540418,
  'style is formal': 761.210213106432,
  'different sentence structures': 700.7600744275024,
  'technical language': 691.8708888939133,
  'consistent verb tense': 689.8113988590511,
  'complex sentence structures': 665.8148527452879,
  'casual language': 525.8945019386799,
  'tense usage': 462.5896328014582,
  'formal tone': 458.4649270811825},
 0: {'technical language': 232.9070319048817,
  'style is professional': 223.8015787285069,
  'style is formal': 212.63853370813473,
  'casual language': 208.37329322098637,
  'diverse sentence structures': 120.59621956909814,
  'formal tone': 99.66628849590924,
  'consistent verb tense': 96.52650973795595,
  'specialized language': 95.23352668654036,
  'simple sentence structures': 94.73339637634739,
  'complex sentence structures': 91.67016088522081},
 1: {'intricate sentence structures': 5.064023167514317,
  'formal tone': 2.49165721239773

In [81]:
json.dump(clusters_and_top10_feats, open('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability-w-gram2vec/clusters_w_top10_feats.json', 'w'))

### Generated Explanations:

In [17]:
test_df = pd.read_json('/mnt/swordfish-pool2/milad/hiatus-data/phase_1/explainability/test_authors_explained.json')

In [22]:
test_df[['documentID', 'documetn_style_description']].head()

,documentID,documetn_style_description
0,00491e44-990a-5fa5-8ab1-ad9e86b2edd4,"[critical thinking, complex sentence structures, figurative language, rhetorical questions, informal language, style is analytical, style is persuasive, rhetorical questions, technical language, style is authoritative, style is user, concise problem statement, analytical thought, speculative methods, technical descriptions]"
1,006dcd64-cb43-5250-bd83-182f17241557,"[figurative language, style is varied, style is vivid, diverse sentence structures, style is casual, style is vividly descriptive, active voice, concise sentences, inclusive writing style, style is descriptive, reflective tone, parallel structures, vivid descriptions, evaluation is unbiased, emotional expression]"
2,00c44d94-19ad-534d-bab7-f9a9f93fc43f,"[concise writing, style is speculative, tone is neutral, style is informative, investigative writing style, multiple perspectives, verb tenses, descriptive storytelling, style is contemplative, author is knowledgeable, diverse subjects, impactful repetition, approach is organized, technical information, author is engaging]"
3,014a02f1-c239-561a-997c-13e68f3ba5c6,"[style is diverse, proper nouns, diverse sentence structures, subordinate clauses, diverse sentence connectors, emotive language, vivid description, style is succinct, concise sentences, rhetorical questions, temporal effect, encouraging tone, proper nouns, balanced view, style is rhetorical]"
4,01842e69-0101-5395-a3bf-8baf1191f522,"[style is casual, style is vivid, tone is casual, style is straightforward, style is transitional, style is clear, present tense, consistent verb tense, consistent use, style is vivid, concise sentences, tense verbs, style is vivid, style is light, active voice]"
